In [1]:
import requests

headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/tiingo/daily/aapl?token="YOUR_TOKEN"", headers=headers)
print(requestResponse.json())

{'ticker': 'AAPL', 'name': 'Apple Inc', 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers digital content and applications through the iTunes Store, App StoreSM, iBookstoreSM, and Mac App Store. The Company distributes its products worldwide through its retail stores, online stores, and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers, and value-added resellers. In February 2012, the Company acquired app-search engine Chomp.", 

##  This is Intraday data freq = every 5 MIns

In [7]:
headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/iex/<ticker>/prices?startDate=2024-08-28&endDate=2024-09-23&resampleFreq=5min&afterHours=False&columns=open,high,low,close,volume&token=e9099efb79717f8ff3d48cd67ab8ae71a6f71966", headers=headers)
print(requestResponse.json())

{'detail': 'Not found.'}


In [15]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Tiingo API Key
API_KEY = 'e9099efb79717f8ff3d48cd67ab8ae71a6f71966'  # Replace with your actual API key

In [16]:
def get_trading_data(ticker, start_date, end_date):
    # Construct the API URL
    url = f"https://api.tiingo.com/iex/{ticker}/prices"
    headers = {'Content-Type': 'application/json'}
    params = {
        'token': API_KEY,
        'startDate': start_date,
        'endDate': end_date,
        'resampleFreq': '5min',
        'afterHours': 'false',
        columns=open,high,low,close,volume
    }

    # Make the request
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data

def filter_trading_hours(data):
    trading_data = []
    for item in data:
        # Parse and convert the timestamp to datetime object with timezone info
        timestamp = datetime.fromisoformat(item['date'].replace('Z', '+00:00')).astimezone()
        
        # Filter for active trading hours (9:30 AM to 4:00 PM New York time)
        if timestamp.hour >= 9 and timestamp.minute >= 30 and timestamp.hour < 16:
            # Remove timezone information
            item['date'] = timestamp.replace(tzinfo=None).isoformat()
            trading_data.append(item)
    return trading_data

def get_last_n_trading_days(ticker, n_days):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=10)  # Look back 10 days to ensure 3 trading days are covered

    raw_data = get_trading_data(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    filtered_data = filter_trading_hours(raw_data)

    # Convert to DataFrame
    df = pd.DataFrame(filtered_data)
    df['date'] = pd.to_datetime(df['date'])

    # Ensure only the last n trading days are included
    trading_days = df['date'].dt.date.unique()[-n_days:]
    df = df[df['date'].dt.date.isin(trading_days)]

    # Remove timezone information if any remains
    df['date'] = df['date'].dt.tz_localize(None)
    
    return df


In [17]:
def save_to_excel(df, filename):
    # Save DataFrame to Excel
    df.to_excel(filename, index=False)

# Get last 3 active trading days for AAPL
df_aapl = get_last_n_trading_days('aapl', 3)
df_aapl.head()
# Save to Excel
save_to_excel(df_aapl, 'AAPL_Last_3_Trading_Days.xlsx')


In [18]:
print(data)

[{'date': '2024-09-03T13:30:00.000Z', 'close': 227.015, 'high': 229.0, 'low': 226.925, 'open': 228.615}, {'date': '2024-09-03T13:35:00.000Z', 'close': 227.09, 'high': 227.685, 'low': 226.625, 'open': 227.05}, {'date': '2024-09-03T13:40:00.000Z', 'close': 226.035, 'high': 227.155, 'low': 226.035, 'open': 227.07}, {'date': '2024-09-03T13:45:00.000Z', 'close': 226.135, 'high': 226.14, 'low': 225.315, 'open': 226.035}, {'date': '2024-09-03T13:50:00.000Z', 'close': 225.65, 'high': 226.715, 'low': 225.55, 'open': 226.125}, {'date': '2024-09-03T13:55:00.000Z', 'close': 226.17, 'high': 226.24, 'low': 225.52, 'open': 225.54}, {'date': '2024-09-03T14:00:00.000Z', 'close': 225.94, 'high': 226.49, 'low': 225.56, 'open': 225.91}, {'date': '2024-09-03T14:05:00.000Z', 'close': 225.535, 'high': 226.27, 'low': 225.39, 'open': 225.96}, {'date': '2024-09-03T14:10:00.000Z', 'close': 224.875, 'high': 225.595, 'low': 224.86, 'open': 225.49}, {'date': '2024-09-03T14:15:00.000Z', 'close': 225.23, 'high': 225.